In [31]:
#!pip install scikit-surprise
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

###############################################################################
# pemalink: https://github.com/JossueGG/tec_bigdata_equipo17/blob/30ae4d31c8f6bf2b6be5bf47f2294a16199b19c4/ratings_small.csv

# URL del archivo CSV en git o un contenedor
url = 'https://raw.githubusercontent.com/JossueGG/tec_bigdata_equipo17/30ae4d31c8f6bf2b6be5bf47f2294a16199b19c4/ratings_small.csv?token=GHSAT0AAAAAACSA6SWLVU4K6DI23MDJXKEGZR6R7OQ'

# Rango de las calificaciones
reader = Reader(rating_scale=(1, 5))

# Número de recomendaciones a desplegar
rec = 5

###############################################################################

# Cargar los datos desde la URL
df = pd.read_csv(url)

# Crear un Dataset de Surprise
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Entrenar el modelo SVD
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Obtener el nombre de la película a partir de su ID
movie_titles = df[['movieId', 'title']].drop_duplicates().set_index('movieId')['title'].to_dict()

# Función para obtener las mejores recomendaciones para un usuario dado
def get_top_recommendations(user_id, n=rec):
    items = []
    # Obtener todos los ítems que el usuario no ha calificado
    items_not_rated_by_user = [item for item in range(1, trainset.n_items + 1)
                               if item not in trainset.ur[user_id]]
    # Obtener la predicción de calificación para cada ítem no calificado
    for item_id in items_not_rated_by_user:
        predicted_rating = model.predict(user_id, item_id).est
        items.append((item_id, predicted_rating))
    # Ordenar las predicciones y devolver las mejores n recomendaciones
    top_items = sorted(items, key=lambda x: x[1], reverse=True)[:n]
    return [(item[0], movie_titles[item[0]]) for item in top_items]


###############################################################################
# Ejemplo de uso, recomendación para el usuario dado
###############################################################################
user_id = 1
top_recommendations = get_top_recommendations(user_id)
print("Top recomendaciones para el usuario:", user_id)
for rank, (movie_id, title) in enumerate(top_recommendations, 1):
    print(f"{rank}. {title} (ID: {movie_id})")

# Evaluar el modelo con 5-fold cross-validation
print("\n\n\nEvaluación del modelo:")
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Top recomendaciones para el usuario: 1
1. The African Queen (ID: 969)
2. Monty Python and the Holy Grail (ID: 1136)
3. Cinema Paradiso (ID: 1172)
4. Chinatown (ID: 1252)
5. Amélie (ID: 4973)



Evaluación del modelo:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8941  0.8912  0.8973  0.9082  0.8952  0.8972  0.0058  
MAE (testset)     0.6895  0.6838  0.6924  0.6988  0.6888  0.6907  0.0049  
Fit time          1.84    1.90    2.23    1.82    1.86    1.93    0.15    
Test time         0.12    0.43    0.13    0.27    0.23    0.24    0.11    
